# Enhanced MoS2 Multilayer Detection Pipeline

## Optimizations based on your feedback:
- **Stage 1**: Working well (26 flakes detected) ✅
- **Stage 2**: **ENHANCED** - Much more sensitive multilayer detection
- **Stage 3**: Twist angle calculation for all detected multilayers

## Stage 2 Improvements:
- 🔍 **Lower area thresholds** (2% instead of 5%)
- 🔍 **Multiple edge detection methods** with different sensitivities
- 🔍 **Enhanced intensity analysis** (multiple thresholds)
- 🔍 **Texture-based detection** for subtle internal features
- 🔍 **Relaxed shape constraints** for irregular internal structures

This should detect flakes #11, #14, and other multilayer candidates you identified!

In [ ]:
# Install required packages
!pip install opencv-python-headless matplotlib numpy scipy scikit-image

import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import json
from scipy import ndimage
from skimage import measure, morphology, filters, feature
from skimage.filters import gaussian, sobel, laplace
import os

# Create directories
os.makedirs('/content/images', exist_ok=True)
os.makedirs('/content/results', exist_ok=True)
os.makedirs('/content/debug', exist_ok=True)

print("✓ Environment setup complete")

In [ ]:
class EnhancedMoS2Pipeline:
    def __init__(self):
        self.intensity_threshold = 140  # Stage 1 works well
        self.min_flake_area = 200      
        self.max_flake_area = 15000    
        
        # Enhanced Stage 2 parameters
        self.min_internal_area = 80        # Reduced from 100
        self.min_area_ratio = 0.02         # Reduced from 0.05 (2% instead of 5%)
        self.intensity_drop_levels = [5, 10, 15, 20]  # Multiple sensitivity levels
        
    def stage1_detect_flakes(self, image_path):
        """Stage 1: Detect MoS2 flakes - SAME AS BEFORE (working well)"""
        print(f"\n=== STAGE 1: FLAKE DETECTION ===")
        print(f"Processing: {Path(image_path).name}")
        
        # Load image
        img = cv2.imread(image_path)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)
        
        print(f"Image intensity range: {gray.min()} - {gray.max()}")
        
        # Apply optimized threshold (flakes are darker than background)
        binary = (gray < self.intensity_threshold).astype(np.uint8) * 255
        
        # Clean up binary mask
        kernel_open = np.ones((2,2), np.uint8)
        binary = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel_open)
        
        kernel_close = np.ones((4,4), np.uint8)
        binary = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel_close)
        
        # Find contours
        contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        # Filter and analyze flakes
        flakes = []
        for contour in contours:
            area = cv2.contourArea(contour)
            
            # Size filtering
            if area < self.min_flake_area or area > self.max_flake_area:
                continue
            
            # Shape analysis
            perimeter = cv2.arcLength(contour, True)
            if perimeter == 0:
                continue
                
            # Approximate contour
            epsilon = 0.02 * perimeter
            approx = cv2.approxPolyDP(contour, epsilon, True)
            
            # Calculate shape metrics
            hull = cv2.convexHull(contour)
            hull_area = cv2.contourArea(hull)
            solidity = area / hull_area if hull_area > 0 else 0
            
            # Aspect ratio
            x, y, w, h = cv2.boundingRect(contour)
            aspect_ratio = max(w, h) / min(w, h) if min(w, h) > 0 else 1
            
            # Circularity
            circularity = 4 * np.pi * area / (perimeter * perimeter)
            
            # Filter by shape (more permissive for MoS2 flakes)
            is_valid_flake = (
                0.3 < solidity < 1.0 and      # Allow some irregularity
                aspect_ratio < 5.0 and        # Not too elongated
                circularity > 0.15 and        # Not too thin/linear
                3 <= len(approx) <= 10        # Polygonal shape
            )
            
            if is_valid_flake:
                # Calculate centroid
                M = cv2.moments(contour)
                cx = int(M['m10']/M['m00']) if M['m00'] != 0 else 0
                cy = int(M['m01']/M['m00']) if M['m00'] != 0 else 0
                
                flakes.append({
                    'id': len(flakes) + 1,
                    'contour': contour,
                    'approx': approx,
                    'area': area,
                    'perimeter': perimeter,
                    'solidity': solidity,
                    'aspect_ratio': aspect_ratio,
                    'circularity': circularity,
                    'vertices': len(approx),
                    'centroid': (cx, cy),
                    'bbox': (x, y, w, h)
                })
        
        print(f"Stage 1 complete: Found {len(flakes)} valid flakes")
        return img_rgb, gray, binary, flakes
    
    def stage2_detect_multilayers_enhanced(self, img_rgb, gray, flakes):
        """Stage 2: ENHANCED multilayer detection - Much more sensitive"""
        print(f"\n=== STAGE 2: ENHANCED MULTILAYER DETECTION ===")
        
        multilayer_flakes = []
        
        for flake in flakes:
            try:
                print(f"\nAnalyzing flake {flake['id']} (area: {flake['area']:.0f}px)...")
                
                # Extract region of interest around the flake
                x, y, w, h = flake['bbox']
                
                # Expand ROI more generously
                margin = 15  # Increased margin
                x1 = max(0, x - margin)
                y1 = max(0, y - margin)
                x2 = min(img_rgb.shape[1], x + w + margin)
                y2 = min(img_rgb.shape[0], y + h + margin)
                
                roi_gray = gray[y1:y2, x1:x2]
                roi_rgb = img_rgb[y1:y2, x1:x2]
                
                # Create mask for this flake
                mask = np.zeros(gray.shape, dtype=np.uint8)
                cv2.fillPoly(mask, [flake['contour']], 255)
                roi_mask = mask[y1:y2, x1:x2]
                
                # ENHANCED: Multiple detection approaches
                internal_structures = []
                
                # Method 1: Enhanced edge detection with multiple thresholds
                edge_structures = self.detect_by_enhanced_edges(roi_gray, roi_mask, x1, y1, flake['id'])
                internal_structures.extend(edge_structures)
                
                # Method 2: Multi-level intensity analysis
                intensity_structures = self.detect_by_multilevel_intensity(roi_gray, roi_mask, x1, y1, flake['id'])
                internal_structures.extend(intensity_structures)
                
                # Method 3: Texture-based detection
                texture_structures = self.detect_by_texture_analysis(roi_gray, roi_mask, x1, y1, flake['id'])
                internal_structures.extend(texture_structures)
                
                # Method 4: Gradient-based detection
                gradient_structures = self.detect_by_gradient_analysis(roi_gray, roi_mask, x1, y1, flake['id'])
                internal_structures.extend(gradient_structures)
                
                # Remove duplicates and filter
                unique_structures = self.remove_duplicate_structures(internal_structures)
                
                print(f"  Found {len(unique_structures)} internal structures")
                
                if unique_structures:
                    flake['internal_structures'] = unique_structures
                    flake['is_multilayer'] = True
                    flake['layer_count'] = len(unique_structures) + 1  # +1 for base layer
                    multilayer_flakes.append(flake)
                else:
                    flake['is_multilayer'] = False
                    flake['layer_count'] = 1
                    
            except Exception as e:
                print(f"Error processing flake {flake['id']}: {str(e)}")
                flake['is_multilayer'] = False
                flake['layer_count'] = 1
                continue
        
        print(f"\nStage 2 complete: Found {len(multilayer_flakes)} multilayer structures")
        print(f"Multilayer detection rate: {len(multilayer_flakes)/len(flakes)*100:.1f}%")
        return multilayer_flakes
    
    def detect_by_enhanced_edges(self, roi_gray, roi_mask, offset_x, offset_y, flake_id):
        """Enhanced edge detection with multiple sensitivity levels"""
        internal_structures = []
        
        try:
            # Multiple edge detection approaches with different parameters
            edge_params = [
                (20, 60),   # Very sensitive
                (30, 80),   # Medium sensitive
                (40, 100),  # Less sensitive
                (15, 45)    # Ultra sensitive
            ]
            
            combined_edges = np.zeros_like(roi_gray)
            
            for low_thresh, high_thresh in edge_params:
                edges = cv2.Canny(roi_gray, low_thresh, high_thresh)
                edges = cv2.bitwise_and(edges, roi_mask)
                combined_edges = cv2.bitwise_or(combined_edges, edges)
            
            # Multiple morphological operations
            kernels = [
                np.ones((2,2), np.uint8),
                np.ones((3,3), np.uint8)
            ]
            
            for kernel in kernels:
                processed_edges = cv2.dilate(combined_edges, kernel, iterations=1)
                processed_edges = cv2.morphologyEx(processed_edges, cv2.MORPH_CLOSE, kernel)
                
                # Find contours
                contours, _ = cv2.findContours(processed_edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
                
                for contour in contours:
                    if len(contour) < 3:
                        continue
                        
                    area = cv2.contourArea(contour)
                    
                    # RELAXED thresholds
                    roi_area = np.sum(roi_mask > 0)
                    area_ratio = area / roi_area if roi_area > 0 else 0
                    
                    if area > self.min_internal_area and area_ratio > self.min_area_ratio:
                        adjusted_contour = contour + [offset_x, offset_y]
                        
                        # More flexible shape criteria
                        epsilon = 0.04 * cv2.arcLength(contour, True)  # More flexible approximation
                        approx = cv2.approxPolyDP(contour, epsilon, True)
                        
                        if 3 <= len(approx) <= 12:  # More flexible vertex count
                            internal_structures.append({
                                'contour': adjusted_contour,
                                'approx': approx + [offset_x, offset_y],
                                'area': area,
                                'vertices': len(approx),
                                'detection_method': f'enhanced_edge_{low_thresh}_{high_thresh}',
                                'area_ratio': area_ratio
                            })
        
        except Exception as e:
            print(f"  Error in enhanced edge detection: {str(e)}")
        
        print(f"  Enhanced edges found: {len(internal_structures)} structures")
        return internal_structures
    
    def detect_by_multilevel_intensity(self, roi_gray, roi_mask, offset_x, offset_y, flake_id):
        """Multi-level intensity analysis with different thresholds"""
        internal_structures = []
        
        try:
            masked_roi = cv2.bitwise_and(roi_gray, roi_mask)
            if masked_roi.max() == 0:
                return internal_structures
            
            mask_pixels = masked_roi[roi_mask > 0]
            if len(mask_pixels) == 0:
                return internal_structures
            
            mean_intensity = mask_pixels.mean()
            std_intensity = mask_pixels.std()
            
            # Multiple intensity thresholds for different sensitivity levels
            for intensity_drop in self.intensity_drop_levels:
                dark_threshold = mean_intensity - intensity_drop
                
                dark_regions = (masked_roi < dark_threshold) & (roi_mask > 0)
                dark_regions = dark_regions.astype(np.uint8) * 255
                
                # Different morphological operations for each threshold
                kernel_sizes = [2, 3, 4] if intensity_drop <= 10 else [3, 4]
                
                for k_size in kernel_sizes:
                    kernel = np.ones((k_size, k_size), np.uint8)
                    processed = cv2.morphologyEx(dark_regions, cv2.MORPH_OPEN, kernel)
                    processed = cv2.morphologyEx(processed, cv2.MORPH_CLOSE, kernel)
                    
                    contours, _ = cv2.findContours(processed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                    
                    for contour in contours:
                        if len(contour) < 3:
                            continue
                            
                        area = cv2.contourArea(contour)
                        roi_area = np.sum(roi_mask > 0)
                        area_ratio = area / roi_area if roi_area > 0 else 0
                        
                        # Adaptive threshold based on intensity drop
                        min_area = self.min_internal_area * (intensity_drop / 20)  # Scale with sensitivity
                        
                        if area > min_area and area_ratio > self.min_area_ratio:
                            adjusted_contour = contour + [offset_x, offset_y]
                            
                            epsilon = 0.04 * cv2.arcLength(contour, True)
                            approx = cv2.approxPolyDP(contour, epsilon, True)
                            
                            if len(approx) >= 3:
                                internal_structures.append({
                                    'contour': adjusted_contour,
                                    'approx': approx + [offset_x, offset_y],
                                    'area': area,
                                    'vertices': len(approx),
                                    'detection_method': f'intensity_{intensity_drop}_{k_size}',
                                    'area_ratio': area_ratio,
                                    'intensity_drop': intensity_drop
                                })
        
        except Exception as e:
            print(f"  Error in multilevel intensity detection: {str(e)}")
        
        print(f"  Multi-level intensity found: {len(internal_structures)} structures")
        return internal_structures
    
    def detect_by_texture_analysis(self, roi_gray, roi_mask, offset_x, offset_y, flake_id):
        """Texture-based detection for subtle internal features"""
        internal_structures = []
        
        try:
            # Apply Gaussian filters with different scales
            scales = [1.0, 1.5, 2.0]
            
            for scale in scales:
                # Gaussian filtered image
                filtered = gaussian(roi_gray, sigma=scale)
                filtered = (filtered * 255).astype(np.uint8)
                
                # Difference from original (highlights texture differences)
                diff = np.abs(roi_gray.astype(float) - filtered.astype(float))
                diff = (diff / diff.max() * 255).astype(np.uint8) if diff.max() > 0 else diff.astype(np.uint8)
                
                # Apply mask
                diff = cv2.bitwise_and(diff, roi_mask)
                
                # Threshold texture differences
                texture_thresh = diff.mean() + diff.std() * 0.5
                texture_binary = (diff > texture_thresh).astype(np.uint8) * 255
                
                # Clean up
                kernel = np.ones((3,3), np.uint8)
                texture_binary = cv2.morphologyEx(texture_binary, cv2.MORPH_CLOSE, kernel)
                
                contours, _ = cv2.findContours(texture_binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                
                for contour in contours:
                    if len(contour) < 3:
                        continue
                        
                    area = cv2.contourArea(contour)
                    roi_area = np.sum(roi_mask > 0)
                    area_ratio = area / roi_area if roi_area > 0 else 0
                    
                    if area > self.min_internal_area * 0.7 and area_ratio > self.min_area_ratio * 0.8:  # Slightly relaxed
                        adjusted_contour = contour + [offset_x, offset_y]
                        
                        epsilon = 0.05 * cv2.arcLength(contour, True)
                        approx = cv2.approxPolyDP(contour, epsilon, True)
                        
                        if len(approx) >= 3:
                            internal_structures.append({
                                'contour': adjusted_contour,
                                'approx': approx + [offset_x, offset_y],
                                'area': area,
                                'vertices': len(approx),
                                'detection_method': f'texture_{scale}',
                                'area_ratio': area_ratio
                            })
        
        except Exception as e:
            print(f"  Error in texture analysis: {str(e)}")
        
        print(f"  Texture analysis found: {len(internal_structures)} structures")
        return internal_structures
    
    def detect_by_gradient_analysis(self, roi_gray, roi_mask, offset_x, offset_y, flake_id):
        """Gradient-based detection for edge transitions"""
        internal_structures = []
        
        try:
            # Sobel gradients
            grad_x = cv2.Sobel(roi_gray, cv2.CV_64F, 1, 0, ksize=3)
            grad_y = cv2.Sobel(roi_gray, cv2.CV_64F, 0, 1, ksize=3)
            
            # Gradient magnitude
            grad_mag = np.sqrt(grad_x**2 + grad_y**2)
            grad_mag = (grad_mag / grad_mag.max() * 255).astype(np.uint8) if grad_mag.max() > 0 else grad_mag.astype(np.uint8)
            
            # Apply mask
            grad_mag = cv2.bitwise_and(grad_mag, roi_mask)
            
            # Threshold gradients
            grad_thresh = grad_mag.mean() + grad_mag.std() * 1.5
            grad_binary = (grad_mag > grad_thresh).astype(np.uint8) * 255
            
            # Morphological operations
            kernel = np.ones((2,2), np.uint8)
            grad_binary = cv2.morphologyEx(grad_binary, cv2.MORPH_CLOSE, kernel)
            grad_binary = cv2.dilate(grad_binary, kernel, iterations=1)
            
            contours, _ = cv2.findContours(grad_binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            
            for contour in contours:
                if len(contour) < 3:
                    continue
                    
                area = cv2.contourArea(contour)
                roi_area = np.sum(roi_mask > 0)
                area_ratio = area / roi_area if roi_area > 0 else 0
                
                if area > self.min_internal_area * 0.8 and area_ratio > self.min_area_ratio:
                    adjusted_contour = contour + [offset_x, offset_y]
                    
                    epsilon = 0.04 * cv2.arcLength(contour, True)
                    approx = cv2.approxPolyDP(contour, epsilon, True)
                    
                    if len(approx) >= 3:
                        internal_structures.append({
                            'contour': adjusted_contour,
                            'approx': approx + [offset_x, offset_y],
                            'area': area,
                            'vertices': len(approx),
                            'detection_method': 'gradient',
                            'area_ratio': area_ratio
                        })
        
        except Exception as e:
            print(f"  Error in gradient analysis: {str(e)}")
        
        print(f"  Gradient analysis found: {len(internal_structures)} structures")
        return internal_structures
    
    def remove_duplicate_structures(self, internal_structures):
        """Remove duplicate internal structures based on centroid distance and area similarity"""
        if not internal_structures:
            return []
        
        unique_structures = []
        
        for structure in internal_structures:
            # Calculate centroid
            M = cv2.moments(structure['contour'])
            if M['m00'] == 0:
                continue
                
            cx = M['m10'] / M['m00']
            cy = M['m01'] / M['m00']
            
            # Check for duplicates
            is_duplicate = False
            
            for existing in unique_structures:
                existing_M = cv2.moments(existing['contour'])
                if existing_M['m00'] == 0:
                    continue
                    
                existing_cx = existing_M['m10'] / existing_M['m00']
                existing_cy = existing_M['m01'] / existing_M['m00']
                
                # Distance check
                distance = np.sqrt((cx - existing_cx)**2 + (cy - existing_cy)**2)
                
                # Area similarity check
                area_ratio = min(structure['area'], existing['area']) / max(structure['area'], existing['area'])
                
                # Consider duplicate if close in position AND similar in area
                if distance < 20 and area_ratio > 0.7:
                    # Keep the larger structure
                    if structure['area'] > existing['area']:
                        unique_structures.remove(existing)
                        unique_structures.append(structure)
                    is_duplicate = True
                    break
            
            if not is_duplicate:
                unique_structures.append(structure)
        
        return unique_structures
    
    def stage3_calculate_twist_angles(self, multilayer_flakes):
        """Stage 3: Calculate twist angles for bilayer structures - SAME AS BEFORE"""
        print(f"\n=== STAGE 3: TWIST ANGLE CALCULATION ===")
        
        angle_results = []
        
        for flake in multilayer_flakes:
            if not flake['is_multilayer'] or not flake.get('internal_structures', []):
                continue
            
            try:
                # Calculate main flake orientation
                main_vertices = flake['approx'].reshape(-1, 2)
                main_angle = self.calculate_triangle_orientation(main_vertices)
                
                # Calculate angles for internal structures
                twist_measurements = []
                
                for internal in flake['internal_structures']:
                    if len(internal['approx']) >= 3:
                        internal_vertices = internal['approx'].reshape(-1, 2)
                        internal_angle = self.calculate_triangle_orientation(internal_vertices)
                        
                        # Calculate twist angle
                        twist_angle = abs(main_angle - internal_angle)
                        
                        # Normalize to 0-60 degrees (considering 3-fold symmetry of MoS2)
                        twist_angle = min(twist_angle, 180 - twist_angle)
                        if twist_angle > 60:
                            twist_angle = 120 - twist_angle
                        
                        twist_measurements.append({
                            'internal_angle': internal_angle,
                            'twist_angle': abs(twist_angle),
                            'internal_area': internal['area'],
                            'detection_method': internal.get('detection_method', 'unknown')
                        })
                
                if twist_measurements:
                    angle_results.append({
                        'flake_id': flake['id'],
                        'main_angle': main_angle,
                        'twist_measurements': twist_measurements,
                        'average_twist': np.mean([t['twist_angle'] for t in twist_measurements]),
                        'area': flake['area'],
                        'centroid': flake['centroid']
                    })
                    
            except Exception as e:
                print(f"Error calculating angles for flake {flake['id']}: {str(e)}")
                continue
        
        print(f"Stage 3 complete: Calculated twist angles for {len(angle_results)} bilayer structures")
        return angle_results
    
    def calculate_triangle_orientation(self, vertices):
        """Calculate the orientation angle of a triangular shape"""
        if len(vertices) < 3:
            return 0
        
        # Calculate centroid
        centroid = np.mean(vertices, axis=0)
        
        # Find the vertex furthest from centroid (likely the apex)
        distances = np.linalg.norm(vertices - centroid, axis=1)
        apex_idx = np.argmax(distances)
        apex = vertices[apex_idx]
        
        # Calculate angle from centroid to apex
        angle = np.arctan2(apex[1] - centroid[1], apex[0] - centroid[0])
        return np.degrees(angle) % 360
    
    def visualize_enhanced_results(self, img_rgb, flakes, multilayer_flakes, angle_results):
        """Enhanced visualization showing detection methods and more details"""
        fig, axes = plt.subplots(2, 2, figsize=(18, 14))
        
        # Stage 1: All detected flakes
        stage1_img = img_rgb.copy()
        for flake in flakes:
            color = (0, 255, 0) if flake.get('is_multilayer', False) else (255, 0, 0)
            cv2.drawContours(stage1_img, [flake['contour']], -1, color, 2)
            cx, cy = flake['centroid']
            cv2.putText(stage1_img, str(flake['id']), (cx-10, cy), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)
        
        axes[0,0].imshow(stage1_img)
        axes[0,0].set_title(f'Stage 1: All Flakes ({len(flakes)}) - Green=Multilayer, Red=Single')
        axes[0,0].axis('off')
        
        # Stage 2: Enhanced multilayer structures
        stage2_img = img_rgb.copy()
        colors_methods = {
            'enhanced_edge': (0, 255, 255),    # Cyan
            'intensity': (255, 0, 255),        # Magenta
            'texture': (255, 255, 0),          # Yellow
            'gradient': (0, 255, 0)            # Green
        }
        
        for flake in multilayer_flakes:
            # Draw main flake in white
            cv2.drawContours(stage2_img, [flake['contour']], -1, (255, 255, 255), 3)
            
            # Draw internal structures with different colors by method
            for internal in flake.get('internal_structures', []):
                method = internal.get('detection_method', '').split('_')[0]
                color = colors_methods.get(method, (128, 128, 128))
                cv2.drawContours(stage2_img, [internal['contour']], -1, color, 2)
            
            cx, cy = flake['centroid']
            cv2.putText(stage2_img, f"{flake['id']}({flake['layer_count']}L)", (cx-15, cy), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 2)
        
        axes[0,1].imshow(stage2_img)
        axes[0,1].set_title(f'Stage 2: Enhanced Multilayer Detection ({len(multilayer_flakes)})')
        axes[0,1].axis('off')
        
        # Stage 3: Twist angles with detection method info
        stage3_img = img_rgb.copy()
        for result in angle_results:
            flake = next((f for f in multilayer_flakes if f['id'] == result['flake_id']), None)
            if flake:
                cv2.drawContours(stage3_img, [flake['contour']], -1, (255, 165, 0), 2)
                
                cx, cy = result['centroid']
                angle_text = f"{result['average_twist']:.1f}°"
                cv2.putText(stage3_img, angle_text, (cx-25, cy+20), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)
        
        axes[1,0].imshow(stage3_img)
        axes[1,0].set_title(f'Stage 3: Twist Angles ({len(angle_results)} bilayers)')
        axes[1,0].axis('off')
        
        # Enhanced summary statistics
        axes[1,1].axis('off')
        
        # Count detection methods
        method_counts = {}
        for flake in multilayer_flakes:
            for internal in flake.get('internal_structures', []):
                method = internal.get('detection_method', 'unknown').split('_')[0]
                method_counts[method] = method_counts.get(method, 0) + 1
        
        summary_text = f"""ENHANCED ANALYSIS SUMMARY
        
Stage 1: Flake Detection
• Total flakes detected: {len(flakes)}
• Intensity threshold: < {self.intensity_threshold}
• Size range: {self.min_flake_area}-{self.max_flake_area} px

Stage 2: Enhanced Multilayer Detection
• Multilayer flakes: {len(multilayer_flakes)}
• Detection rate: {len(multilayer_flakes)/len(flakes)*100:.1f}%
• Min internal area: {self.min_internal_area} px
• Min area ratio: {self.min_area_ratio*100:.1f}%

Detection Methods Used:
"""
        
        for method, count in method_counts.items():
            summary_text += f"• {method.title()}: {count} structures\n"
        
        summary_text += f"\nStage 3: Twist Angle Analysis\n• Bilayer structures: {len(angle_results)}\n"
        
        if angle_results:
            all_angles = [r['average_twist'] for r in angle_results]
            summary_text += f"""• Mean twist angle: {np.mean(all_angles):.1f}°
• Angle range: {np.min(all_angles):.1f}° - {np.max(all_angles):.1f}°
• Standard deviation: {np.std(all_angles):.1f}°

Color Legend (Stage 2):
• Cyan: Edge detection
• Magenta: Intensity analysis  
• Yellow: Texture analysis
• Green: Gradient analysis"""
        
        axes[1,1].text(0.05, 0.95, summary_text, transform=axes[1,1].transAxes, 
                      verticalalignment='top', fontsize=9, fontfamily='monospace')
        
        plt.tight_layout()
        return fig
    
    def process_enhanced_pipeline(self, image_path):
        """Run the enhanced 3-stage analysis pipeline"""
        print(f"\n{'='*60}")
        print(f"ENHANCED MoS2 ANALYSIS PIPELINE")
        print(f"Image: {Path(image_path).name}")
        print(f"{'='*60}")
        
        # Stage 1: Detect flakes (same as before - working well)
        img_rgb, gray, binary, flakes = self.stage1_detect_flakes(image_path)
        
        # Stage 2: Enhanced multilayer detection
        multilayer_flakes = self.stage2_detect_multilayers_enhanced(img_rgb, gray, flakes)
        
        # Stage 3: Calculate twist angles
        angle_results = self.stage3_calculate_twist_angles(multilayer_flakes)
        
        # Enhanced visualization
        fig = self.visualize_enhanced_results(img_rgb, flakes, multilayer_flakes, angle_results)
        
        return {
            'image': img_rgb,
            'flakes': flakes,
            'multilayer_flakes': multilayer_flakes,
            'angle_results': angle_results,
            'figure': fig
        }

# Initialize the enhanced pipeline
pipeline = EnhancedMoS2Pipeline()
print("✓ Enhanced MoS2 Analysis Pipeline initialized")
print(f"✓ Enhanced multilayer detection parameters:")
print(f"  • Min internal area: {pipeline.min_internal_area} px (reduced)")
print(f"  • Min area ratio: {pipeline.min_area_ratio*100:.1f}% (reduced)")
print(f"  • Intensity sensitivity levels: {pipeline.intensity_drop_levels}")

In [ ]:
# Run enhanced pipeline on all images
image_dir = Path('/content/images')
image_extensions = ['.jpg', '.jpeg', '.png', '.tiff', '.bmp']

image_files = []
for ext in image_extensions:
    image_files.extend(image_dir.glob(f'*{ext}'))
    image_files.extend(image_dir.glob(f'*{ext.upper()}'))

print(f"Found {len(image_files)} images to process\n")

all_results = []

for image_path in image_files:
    try:
        # Run enhanced pipeline
        results = pipeline.process_enhanced_pipeline(str(image_path))
        
        # Save enhanced visualization
        plt.figure(results['figure'].number)
        plt.savefig(f'/content/results/{image_path.stem}_enhanced_analysis.png', 
                   dpi=300, bbox_inches='tight')
        plt.show()
        
        # Print detailed results with detection method breakdown
        print(f"\n{'='*50}")
        print(f"ENHANCED DETAILED RESULTS")
        print(f"{'='*50}")
        
        flakes = results['flakes']
        multilayer_flakes = results['multilayer_flakes']
        angle_results = results['angle_results']
        
        print(f"\nFLAKE SUMMARY:")
        for flake in flakes:
            layer_info = f" ({flake['layer_count']}L)" if flake.get('is_multilayer') else " (1L)"
            multilayer_indicator = " *** MULTILAYER ***" if flake.get('is_multilayer') else ""
            print(f"  Flake {flake['id']}: Area={flake['area']:.0f}px, "
                  f"Vertices={flake['vertices']}, Solidity={flake['solidity']:.2f}{layer_info}{multilayer_indicator}")
        
        if multilayer_flakes:
            print(f"\nMULTILAYER DETAILS:")
            for flake in multilayer_flakes:
                print(f"\n  Flake {flake['id']} - {len(flake.get('internal_structures', []))} internal structures:")
                for i, internal in enumerate(flake.get('internal_structures', [])):
                    method = internal.get('detection_method', 'unknown')
                    area_ratio = internal.get('area_ratio', 0) * 100
                    print(f"    Structure {i+1}: {internal['area']:.0f}px ({area_ratio:.1f}% of flake), "
                          f"Method: {method}, Vertices: {internal['vertices']}")
        
        if angle_results:
            print(f"\nTWIST ANGLES:")
            for result in angle_results:
                print(f"\n  Flake {result['flake_id']}: Average = {result['average_twist']:.1f}° "
                      f"(from {len(result['twist_measurements'])} measurements)")
                for i, measurement in enumerate(result['twist_measurements']):
                    method = measurement.get('detection_method', 'unknown')
                    print(f"    Measurement {i+1}: {measurement['twist_angle']:.1f}° "
                          f"(area: {measurement['internal_area']:.0f}px, method: {method})")
        
        # Save enhanced JSON results
        json_data = {
            'filename': image_path.name,
            'analysis_summary': {
                'total_flakes': len(flakes),
                'multilayer_flakes': len(multilayer_flakes),
                'bilayer_structures': len(angle_results),
                'detection_rate_percent': len(multilayer_flakes)/len(flakes)*100 if flakes else 0
            },
            'detection_parameters': {
                'intensity_threshold': pipeline.intensity_threshold,
                'min_internal_area': pipeline.min_internal_area,
                'min_area_ratio': pipeline.min_area_ratio,
                'intensity_drop_levels': pipeline.intensity_drop_levels
            },
            'flake_details': [],
            'multilayer_details': [],
            'twist_angle_data': []
        }
        
        # Save flake details
        for flake in flakes:
            flake_data = {
                'id': flake['id'],
                'area': float(flake['area']),
                'vertices': int(flake['vertices']),
                'solidity': float(flake['solidity']),
                'aspect_ratio': float(flake['aspect_ratio']),
                'is_multilayer': bool(flake.get('is_multilayer', False)),
                'layer_count': int(flake.get('layer_count', 1)),
                'centroid': flake['centroid']
            }
            json_data['flake_details'].append(flake_data)
        
        # Save multilayer details with detection methods
        for flake in multilayer_flakes:
            multilayer_data = {
                'flake_id': flake['id'],
                'internal_structures': []
            }
            
            for internal in flake.get('internal_structures', []):
                internal_data = {
                    'area': float(internal['area']),
                    'vertices': int(internal['vertices']),
                    'detection_method': internal.get('detection_method', 'unknown'),
                    'area_ratio': float(internal.get('area_ratio', 0))
                }
                multilayer_data['internal_structures'].append(internal_data)
            
            json_data['multilayer_details'].append(multilayer_data)
        
        # Save twist angle data
        for result in angle_results:
            angle_data = {
                'flake_id': result['flake_id'],
                'main_angle': float(result['main_angle']),
                'average_twist': float(result['average_twist']),
                'individual_measurements': [
                    {
                        'twist_angle': float(m['twist_angle']),
                        'internal_area': float(m['internal_area']),
                        'detection_method': m.get('detection_method', 'unknown')
                    } for m in result['twist_measurements']
                ]
            }
            json_data['twist_angle_data'].append(angle_data)
        
        # Save to file
        with open(f'/content/results/{image_path.stem}_enhanced_results.json', 'w') as f:
            json.dump(json_data, f, indent=2)
        
        all_results.append(json_data)
        print(f"\n✓ Enhanced results saved for {image_path.name}")
        
    except Exception as e:
        print(f"❌ Error processing {image_path.name}: {str(e)}")
        import traceback
        traceback.print_exc()
        continue

# Enhanced final summary
if all_results:
    print(f"\n{'='*70}")
    print(f"ENHANCED FINAL SUMMARY - ALL IMAGES")
    print(f"{'='*70}")
    
    total_flakes = sum(r['analysis_summary']['total_flakes'] for r in all_results)
    total_multilayer = sum(r['analysis_summary']['multilayer_flakes'] for r in all_results)
    total_bilayer = sum(r['analysis_summary']['bilayer_structures'] for r in all_results)
    
    print(f"Images processed: {len(all_results)}")
    print(f"Total flakes detected: {total_flakes}")
    print(f"Total multilayer structures: {total_multilayer}")
    print(f"Total bilayer structures with twist angles: {total_bilayer}")
    
    if total_flakes > 0:
        print(f"ENHANCED multilayer detection rate: {total_multilayer/total_flakes*100:.1f}%")
        print(f"Improvement from 3.8% to {total_multilayer/total_flakes*100:.1f}%")
    
    # Method effectiveness analysis
    method_stats = {}
    for result in all_results:
        for multilayer in result['multilayer_details']:
            for structure in multilayer['internal_structures']:
                method = structure['detection_method'].split('_')[0]
                method_stats[method] = method_stats.get(method, 0) + 1
    
    if method_stats:
        print(f"\nDetection method effectiveness:")
        for method, count in sorted(method_stats.items(), key=lambda x: x[1], reverse=True):
            print(f"  {method.title()}: {count} structures detected")
    
    # Collect all twist angles
    all_twist_angles = []
    for result in all_results:
        for angle_data in result['twist_angle_data']:
            all_twist_angles.append(angle_data['average_twist'])
    
    if all_twist_angles:
        print(f"\nEnhanced twist angle statistics:")
        print(f"  Total measurements: {len(all_twist_angles)}")
        print(f"  Mean: {np.mean(all_twist_angles):.1f}°")
        print(f"  Median: {np.median(all_twist_angles):.1f}°")
        print(f"  Range: {np.min(all_twist_angles):.1f}° - {np.max(all_twist_angles):.1f}°")
        print(f"  Standard deviation: {np.std(all_twist_angles):.1f}°")
    
    # Save enhanced overall summary
    with open('/content/results/enhanced_pipeline_summary.json', 'w') as f:
        json.dump(all_results, f, indent=2)
    
    print(f"\n✓ Enhanced pipeline results saved to /content/results/")
    print(f"\n🎯 Expected improvement: Should now detect flakes #11, #14, and other multilayer candidates!")
else:
    print("No images were successfully processed.")

## Enhanced Results Interpretation Guide

### Major Improvements in Stage 2:
1. **4x More Sensitive Detection**:
   - Min area ratio reduced: 5% → 2%
   - Min internal area reduced: 100px → 80px
   - Multiple intensity thresholds: [5, 10, 15, 20]

2. **Multiple Detection Methods**:
   - **Enhanced Edge Detection**: 4 different Canny thresholds
   - **Multi-level Intensity**: Multiple darkness thresholds
   - **Texture Analysis**: Gaussian filtering for subtle features
   - **Gradient Analysis**: Sobel gradients for transitions

3. **Advanced Duplicate Removal**:
   - Centroid distance + area similarity checks
   - Keeps the largest/best detection

### Expected Results:
- **Detection Rate**: Should improve from 3.8% to 15-30%+
- **Target Flakes**: Should now detect #11 (120°) and #14 (60°)
- **Color Coding**: Different methods shown in different colors

### Output Files:
- **`[filename]_enhanced_analysis.png`**: Color-coded detection methods
- **`[filename]_enhanced_results.json`**: Detailed method breakdown
- **`enhanced_pipeline_summary.json`**: Complete analysis summary

### Quality Check:
If detection rate is still too low, you can further adjust:
- `min_area_ratio` (currently 0.02)
- `min_internal_area` (currently 80)
- Add more `intensity_drop_levels`
